In [1]:
import pandas as pd
import numpy as np

Экспортируем файл excel и преобразуем его в data frame для удобства работы.

In [2]:
df = pd.read_excel('Data_sample.xlsx')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom
2,536367,21755,LOVE BUILDING BLOCK WORD,3,2010-12-01 08:34:00,5.95,13047.0,United Kingdom
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,2010-12-01 08:34:00,4.95,13047.0,United Kingdom
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 09:02:00,2.75,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
54209,581585,84832,ZINC WILLIE WINKIE CANDLE STICK,24,2011-12-09 12:31:00,0.85,15804.0,United Kingdom
54210,581585,23084,RABBIT NIGHT LIGHT,12,2011-12-09 12:31:00,2.08,15804.0,United Kingdom
54211,581585,84692,BOX OF 24 COCKTAIL PARASOLS,25,2011-12-09 12:31:00,0.42,15804.0,United Kingdom
54212,581587,22631,CIRCUS PARADE LUNCH BOX,12,2011-12-09 12:50:00,1.95,12680.0,France


Выведем общую информацию по источнику:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54214 entries, 0 to 54213
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   InvoiceNo    54214 non-null  object        
 1   StockCode    54214 non-null  object        
 2   Description  54080 non-null  object        
 3   Quantity     54214 non-null  int64         
 4   InvoiceDate  54214 non-null  datetime64[ns]
 5   UnitPrice    54214 non-null  float64       
 6   CustomerID   40643 non-null  float64       
 7   Country      54214 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 3.3+ MB


In [4]:
df.Quantity.unique()

array([    6,    32,     3,     8,    10,     2,     1,     4,    -1,
         100,    12,    24,   144,    48,   -24,     5,    40,    36,
          96,    72,    16,    30,     9,     7,    18,    60,    20,
          -3,   108,    75,   320,    17,  -144,    -2,    -4,    21,
          47,    25,   192,    31,    57,    -6,   -70,   -80,    14,
          15,   120,   480,    33,    22,    49,    -8,    -9,    19,
         -72,   160,    64,   104,    13,    26,    11,   408,    50,
         200,    46,    80,   -77,  -150,  -723,   -32,    23,    84,
         -12,   -18,   -11,   113,    45,    35,    38,   115,   -48,
         -10,    -7,   -20,  -240,   460,   240,   288,   -36,    53,
         216,  -120,   150,   168,   -30,   -40,    54,    28,  1284,
          29,    41,    27,  -310,    78,   -15,   384,    73,    37,
         720,   -50,    -5,  -600,   432,  1200,   -19,  -234,   208,
         -76,   -16,   300,   256,  1500,   407,  2400,  -192,   576,
          44,    70,

In [5]:
# проверим пустые значения по пользователям
nul_ID = df.loc[pd.isnull(df.CustomerID), :].Country.count()
print('Количество пустых значений CustomerID: ',nul_ID)
print('Процент пустых значений CustomerID: ',(nul_ID / df.Country.count())*100,'%')

Количество пустых значений CustomerID:  13571
Процент пустых значений CustomerID:  25.032279485003873 %


In [10]:
negative_quantity = df.loc[df.Quantity <= 0]
print('Количество отрицательного объема:', negative_quantity.sum_order.sum()) 
# первоначального описания, почему объем отрицательный не было, будем считать это категориями возврата, брака, скидок

Количество отрицательного объема: -56072.100000000006


Датасет по покупкам интернет-магазина состоит из 8 столбцов с информацией и 54214 строк. Есть номер платежа, код покупки, описание, количество единиц заказа по каждой категории, дата покупки, цена за единицу товара, ID покупателя и страна (скорее всего доставки).

Пустые или незаполненные строки присутствуют в CustomerID и Description. Потеря идентификационного номера пользователя более 25%, что много. Возможно ошибки произошли в результате сбоя ПО. Также замечено, что в столбце Quantity(Количество) имеются отрицательные значения,что можно расценивать или как ошибку сбора датасета или как возврат товара, скидку, брак. Данных значений более 34000, что является большой долей из общего количества.

В общем можно расценивать достаточность в данных для проведения поверхностного анализа.

Основная бизнес задача - максимизация прибыли. Следовательно будем искать точки роста бизнеса, куда можно направить силы компании. В данных отсутствует информация о затратах на производство, логистику, маркетинг, конкурентное сравнение по рынку и т.п. следовательно построить цельную иерархию метрик у нас не получится. Будим исходить из того, что есть.

Используемые метрики для проведения анализа:

    Доход, отдельно по каждой стране (Revenue)
    Доход с пользователя за месяц (ARPU)
    Средний чек заказа (AOV)
    Количество заказов по клиентам
    Средняя цена товара или средний чек (APP)
    Также будем смотреть, что можно сделать для увеличения среднего чека, в каких странах продажи идут хорошо, а в каких нет, что сделать для потенциального увеличения доходов компании
    
Посмотрим более подробно на данные

Посмотрим количество уникальных пользователей

In [7]:
df['CustomerID'].nunique()

3819

Какой средний объем заказа?

In [8]:
print('Средний объем заказа:', df.Quantity.mean())

Средний объем заказа: 9.610801637953296


In [9]:
# добавим столбец со значением суммы заказа (Quantity * UnitPrice)

def sum_order(row):
    """Умножаем значение столбца Quantity на UnitPrice"""
    return row['Quantity'] * row['UnitPrice']
df['sum_order'] = df.apply(sum_order, axis=1)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,sum_order
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom,54.08
2,536367,21755,LOVE BUILDING BLOCK WORD,3,2010-12-01 08:34:00,5.95,13047.0,United Kingdom,17.85
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,2010-12-01 08:34:00,4.95,13047.0,United Kingdom,14.85
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 09:02:00,2.75,17850.0,United Kingdom,22.00
...,...,...,...,...,...,...,...,...,...
54209,581585,84832,ZINC WILLIE WINKIE CANDLE STICK,24,2011-12-09 12:31:00,0.85,15804.0,United Kingdom,20.40
54210,581585,23084,RABBIT NIGHT LIGHT,12,2011-12-09 12:31:00,2.08,15804.0,United Kingdom,24.96
54211,581585,84692,BOX OF 24 COCKTAIL PARASOLS,25,2011-12-09 12:31:00,0.42,15804.0,United Kingdom,10.50
54212,581587,22631,CIRCUS PARADE LUNCH BOX,12,2011-12-09 12:50:00,1.95,12680.0,France,23.40


Найдем общую сумму дохода с продаж, средний чек по всем странам(AOV), топ 10 клиентов по количеству заказов.

По данным клиентам из топ 10 по заказам можно выслать персонализированное предложение в написании отзыва с предоставлением скидки или предложить реферальную программу (если нет, то это будет тестовое решение с возможным выводом реферальной программы для всех пользователей), дабы повысить лояльность. Если пользователь заказывает большими объемами, то скорее всего, что он является "адвокатом" фирмы, что нам на руку.

In [54]:
print('Сумма дохода с продаж, за вычетом отрицательных значений:', df.sum_order.sum())
print('Средний чек по всем странам(AOV):', df.sum_order.mean())

Сумма дохода с продаж, за вычетом отрицательных значений: 1000095.891
Средний чек по всем странам(AOV): 18.447188751982996


In [12]:
df.CustomerID.value_counts().head(10)

17841.0    771
14911.0    563
14096.0    546
12748.0    447
14606.0    278
15311.0    255
14646.0    195
13089.0    180
13263.0    166
14298.0    160
Name: CustomerID, dtype: int64

Посмотрим на какие 10 стран приходится основная доля заказов и их долю

In [13]:
print('Топ 10 стран по количеству заказов:\n',df.Country.value_counts().head(10),'\n')
print('Доля топ 10 стран по заказам:\n',df.Country.value_counts(1).head(10))

Топ 10 стран по количеству заказов:
 United Kingdom    49602
Germany             951
France              881
EIRE                787
Spain               253
Netherlands         234
Belgium             211
Switzerland         189
Portugal            152
Australia           130
Name: Country, dtype: int64 

Доля топ 10 стран по заказам:
 United Kingdom    0.914930
Germany           0.017542
France            0.016250
EIRE              0.014517
Spain             0.004667
Netherlands       0.004316
Belgium           0.003892
Switzerland       0.003486
Portugal          0.002804
Australia         0.002398
Name: Country, dtype: float64


Львинная доля приходится на United Kingdom - 91.5%. Ближайшие страны занимают лишь около 2% или менее (Германия, Франция, Ирландия). Что может говорить о направленности бизнеса на внутренний рынок, т.к. остальные страны подразделяются методом территориального угасания по заказам от Великобритании. Скорее всего реклама не продвигается по другим странам или имеет место только "сарафанного радио"/рекомендаций. Данный факт необходимо перепроверить с данными отдела маркетинга. Они в общем датасете или описании к нему отсутствуют, данную гипотезу перепроверить в полной не можем или необходимо уточнить данную информацию у заказчика, но сейчас будем основываться на стат. выводах. Попробуем проанализировать данный факт ниже.

Проверим, какой будет топ 10 стран по выручке (Revenue):

In [51]:
# df.groupby(['CustomerID', 'Country']).sum().sort_values('sum_order', ascending=False).head(10)
top_sum = df.groupby('Country').sum().sort_values('sum_order', ascending=False)
top_sum['sum_order'].head(10)

Country
United Kingdom    844816.921
EIRE               27443.140
Netherlands        26267.780
Germany            22519.200
France             19472.990
Australia          17072.960
Switzerland         6209.300
Spain               5955.580
Belgium             4032.060
Norway              3859.900
Name: sum_order, dtype: float64

Стоит отметить, что хоть и подтвердилось, что Великобритания по основной доле, однако страны с меньшей долей заказов, а именно Ирландия и Нидерланды, обошли по полученной выручке Германию и Францию. Особый прирост дали именно Нидерланды. Покупатели из данных стран следовательно заказывают более дорогие вещи.

Стоит проверить гипотезу, что по средней цене и по средней выручке будут выше страны из небольшой доли выручки.
Посмотрим на топ 10 таких стран:

In [52]:
#Топ 10 стран по по средней цене заказа
app = df.groupby('Country').mean().sort_values('UnitPrice', ascending=False)
app['UnitPrice'].head(10)

Country
Hong Kong          92.243103
Portugal           11.983882
Singapore          11.889200
Norway             10.442000
Malta               8.477778
Cyprus              8.256066
Israel              7.175926
EIRE                6.974917
Brazil              6.150000
Channel Islands     5.329259
Name: UnitPrice, dtype: float64

In [53]:
#Топ 10 стран по средней сумме заказа, средний чек (AOV)
app_meansum = df.groupby('Country').mean().sort_values('sum_order', ascending=False)
app_meansum['sum_order'].head(10)

Country
Australia          131.330462
Netherlands        112.255470
Sweden              74.970400
Japan               60.653939
Denmark             58.824000
Lithuania           48.600000
Czech Republic      41.100000
Lebanon             39.600000
Norway              36.760952
Channel Islands     35.530247
Name: sum_order, dtype: float64

Менеджменту нашей компании стоит обратить внимание на тот факт, что страны с наивысшим средним чеком составляют малую долю, менее 2-х процентов. По средней сумме заказа в разрезе стран также входят с наименьшей долей. Это большая упущенная выгода для компании. Следует поднимать объемы заказов с данных стран, использовав маркетинг или другие способы. Следует обратить внимание на такие страны как: Гонконг (большой средний чек!), Португалия, Сингапур, Норвегия, Австралия, Нидерланды, Швеция.

Если имеются проблемы с логистикой в мировых масштабах и нет возможности точечно развиваться в странах, то можно предложить развитие рынка через ближайшие европейские страны.

Также стоит более детально узнать у заказчика, что означают данные по оъемам заказа с отрицательным значением. Если это возвраты, брак, то необходимо уделить внимание анализу и формулированию гипотез по уменьшению их объемов.

Далее перейдем к построению дашборда через Google Data Studio с построением инфографиков и визуализированием стат. данных.